<a href="https://colab.research.google.com/github/carolinaenriqz/Optimization-Algorithms-for-Uncertain-Knapsack-Problem/blob/1stStageVariables-Reduced/26_08_TFG_CE_GeneracionDatos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## cargar datos de github e importar librerias

In [5]:
!pip -q install XlsxWriter
!pip install mip

!git clone --depth 1 https://github.com/likr/kplib.git

import os
import pandas as pd
from IPython.display import display

BASE_DIR = "/content/kplib/00Uncorrelated"
n_values = ["n00050", "n00100", "n00200", "n00500"]
tipo_instancia = "R01000"

instancias = []
for n in n_values:
    ruta = os.path.join(BASE_DIR, n, tipo_instancia)
    if not os.path.isdir(ruta):
        print(f"Ruta no encontrada: {ruta}")
        continue

    archivos = sorted([f for f in os.listdir(ruta) if f.lower().endswith('.kp')])[:15]

    for archivo in archivos:
        ruta_archivo = os.path.join(ruta, archivo)
        try:
            with open(ruta_archivo, 'r') as f:
                lineas = f.readlines()

            num_items = int(lineas[1].strip())
            capacidad = int(lineas[2].strip())
            items = [tuple(map(int, linea.strip().split()))
                     for linea in lineas[4:] if linea.strip()]

            if len(items) != num_items:
                print(f"Advertencia: en {archivo} se esperaban {num_items} ítems pero se leyeron {len(items)}")
                continue

            valores, pesos = zip(*items)
            instancias.append({
                "n": num_items,
                "Capacidad": capacidad,
                "Utilidades": list(valores),
                "Pesos": list(pesos),
                "Archivo": archivo
            })
        except Exception as e:
            print(f"Error al leer {archivo}: {e}")

df_kplib = pd.DataFrame(instancias)
print(f"Instancias cargadas: {df_kplib.shape}")
df_kplib = df_kplib[["Archivo", "n", "Capacidad", "Utilidades", "Pesos"]]
display(df_kplib.head(10))









   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.1/484.1 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 103.0 MB/s eta 0:00:00
  Created wheel for cffi: filename=cffi-1.15.0-cp312-cp312-linux_x86_64.whl size=400935 sha256=05344cedfc81492f1469f40f8bbb86454821259acab278c8c73df4ad7ef06e38
  Stored in directory: /root/.cache/pip/wheels/b9/d6/15/0950847bf7d74ea5f0380b8b23a1d81b45bdf48488b4b8237a
Successfully built cffi
  Attempting uninstall: cffi
    Found existing installation: cffi 1.17.1
    Uninstalling cffi-1.17.1:
      Successfully uninstalled cffi-1.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pygit2 1.18.2 requires cffi>=1.17.0, but you have cffi 1.15.0 which is incompatible.
rpy2 3.5.17 requires

,Archivo,n,Capacidad,Utilidades,Pesos
0,s000.kp,50,14778,"[845, 758, 421, 259, 512, 405, 784, 304, 477, ...","[804, 448, 81, 321, 508, 933, 110, 552, 707, 5..."
1,s001.kp,50,13598,"[135, 848, 764, 256, 496, 450, 652, 789, 94, 2...","[304, 588, 883, 847, 506, 590, 35, 243, 798, 4..."
2,s002.kp,50,13810,"[957, 948, 57, 85, 836, 736, 670, 309, 606, 60...","[491, 925, 501, 832, 354, 883, 900, 462, 568, ..."
3,s003.kp,50,13596,"[238, 545, 370, 604, 626, 66, 14, 838, 260, 23...","[682, 929, 857, 991, 672, 164, 861, 965, 905, ..."
4,s004.kp,50,11618,"[237, 104, 397, 155, 67, 402, 918, 801, 766, 2...","[281, 535, 472, 343, 998, 196, 413, 203, 633, ..."
5,s005.kp,50,11922,"[623, 742, 796, 943, 740, 923, 30, 466, 944, 6...","[338, 310, 819, 481, 316, 482, 705, 58, 976, 2..."
6,s006.kp,50,12044,"[794, 822, 486, 262, 1, 663, 471, 760, 374, 77...","[881, 526, 117, 663, 313, 197, 484, 139, 210, ..."
7,s007.kp,50,12828,"[324, 151, 651, 73, 536, 366, 58, 508, 38, 434...","[981, 119, 419, 758, 152, 489, 40, 669, 765, 5..."
8,s008.kp,50,10985,"[227, 963, 127, 705, 86, 248, 1000, 210, 642, ...","[703, 950, 844, 504, 198, 151, 529, 510, 72, 9..."
9,s009.kp,50,11487,"[464, 374, 139, 867, 7, 503, 899, 81, 555, 617...","[84, 540, 18, 85, 497, 921, 421, 399, 639, 94,..."


## sp model

In [6]:
seed = 123
n_smpl = 8
sizes = (50, 100)
k = 15

import time
import numpy as np
import pandas as pd
from mip import Model, CONTINUOUS, MAXIMIZE, CBC, OptimizationStatus

def solve_sp(bar_c, pesos, capacidad, pct_desviacion, t, n_smpl, seed=123, verbose=False):
    solver_name = CBC
    time_limit = 360000000
    pct_x1, pct_x2, pct_x3, pct_x4 = 0.25, 0.25, 0.25, 0.25
    n = len(bar_c)
    n_x1, n_x2, n_x3 = (int(round(pct_x1*n)), int(round(pct_x2*n)), int(round(pct_x3*n))); n_x4 = n - (n_x1 + n_x2 + n_x3)
    c1, c2, c3, c4 = (np.array(bar_c[:n_x1], float), np.array(bar_c[n_x1:n_x1+n_x2], float), np.array(bar_c[n_x1+n_x2:n_x1+n_x2+n_x3], float), np.array(bar_c[n_x1+n_x2+n_x3:n], float))
    w1, w2, w3, w4 = (np.array(pesos[:n_x1], float), np.array(pesos[n_x1:n_x1+n_x2], float), np.array(pesos[n_x1+n_x2:n_x1+n_x2+n_x3], float), np.array(pesos[n_x1+n_x2+n_x3:n], float))
    rng = np.random.default_rng(seed)
    def _lh(v, delta):
        low, high = v*(1-delta), v*(1+delta)
        return np.minimum(low, high), np.maximum(low, high)
    low2, high2 = _lh(c2, pct_desviacion); U2_base = rng.uniform(low2, high2, size=(n_smpl, n_x2))
    low3, high3 = _lh(c3, pct_desviacion); U3_base = rng.uniform(low3, high3, size=(n_smpl, n_x3))
    low4, high4 = _lh(c4, pct_desviacion); U4_base = rng.uniform(low4, high4, size=(n_smpl, n_x4))
    mu2 = U2_base.mean(axis=0) if U2_base.size else np.array([], float)
    mu3 = U3_base.mean(axis=0) if U3_base.size else np.array([], float)
    mu4 = U4_base.mean(axis=0) if U4_base.size else np.array([], float)
    tile3 = np.tile(mu3, (n_smpl, 1)) if mu3.size else np.zeros((n_smpl, 0), float)
    tile4 = np.tile(mu4, (n_smpl, 1)) if mu4.size else np.zeros((n_smpl, 0), float)
    S2 = S3 = S4 = n_smpl
    p2 = p3 = p4 = 1.0 / n_smpl
    # ---------- t == 1 ----------
    if t == 1:
        mean2, mean3, mean4 = mu2, mu3, mu4
        c_mean = np.concatenate([c1, mean2, mean3, mean4]).astype(float)
        w_all = np.concatenate([w1, w2, w3, w4]).astype(float)
        m = Model(sense=MAXIMIZE, solver_name=solver_name); m.max_seconds = time_limit
        x = [m.add_var(var_type=CONTINUOUS, lb=0.0, ub=1.0, name=f"x_{j}") for j in range(n)]
        m.objective = sum(c_mean[j] * x[j] for j in range(n))
        m += (sum(w_all[j] * x[j] for j in range(n)) <= capacidad), "cap"
        t0 = time.perf_counter(); status = m.optimize(); t1 = time.perf_counter()
        solved   = status in (OptimizationStatus.OPTIMAL, OptimizationStatus.FEASIBLE)
        obj_val  = m.objective_value if solved else None
        x_opt    = np.array([var.x for var in x]) if solved else None
        if verbose:
            print(f"[MIP] SP1 | status={status} | obj={obj_val} | vars/cons={m.num_cols}/{m.num_rows} | time={t1-t0:.2f}s")
        assert m.num_cols == n and m.num_rows == 1, f"Vars/Cons esperadas {n}/1, got {m.num_cols}/{m.num_rows}"
        return {"t": 1, "status": str(status), "objective": obj_val, "n_vars": m.num_cols, "n_constrs": m.num_rows, "x": x_opt, "model": m}
    # ---------- t == 2 ----------
    elif t == 2:
        c234 = np.concatenate([c2, c3, c4]).astype(float) if (n_x2+n_x3+n_x4)>0 else np.array([], float)
        w2_u = np.concatenate([w2, w3, w4]).astype(float)  if (n_x2+n_x3+n_x4)>0 else np.array([], float)
        U2 = np.concatenate([U2_base, tile3, tile4], axis=1) if c234.size>0 else np.zeros((n_smpl, 0), float)
        n_x2_u = c234.size
        S2 = n_smpl; p2 = 1.0 / n_smpl
        m2 = Model(sense=MAXIMIZE, solver_name=solver_name); m2.max_seconds = time_limit
        x1 = [m2.add_var(var_type=CONTINUOUS, lb=0.0, ub=1.0, name=f"x1_{j}") for j in range(n_x1)]
        x2 = {(s2, j): m2.add_var(var_type=CONTINUOUS, lb=0.0, ub=1.0, name=f"x2_{s2}_{j}") for s2 in range(S2) for j in range(n_x2_u)}
        obj = 0.0
        for j in range(n_x1): obj += c1[j] * x1[j]
        for s2 in range(S2):
            for j in range(n_x2_u):
                obj += (p2 * U2[s2, j]) * x2[(s2, j)]
        m2.objective = obj
        for s2 in range(S2):
            expr = 0.0
            for j in range(n_x1):    expr += w1[j]  * x1[j]
            for j in range(n_x2_u):  expr += w2_u[j] * x2[(s2, j)]
            m2 += (expr <= capacidad), f"cap_{s2}"

        t0 = time.perf_counter(); status = m2.optimize(); t1 = time.perf_counter()
        solved = status in (OptimizationStatus.OPTIMAL, OptimizationStatus.FEASIBLE)
        obj_val = m2.objective_value if solved else None

        if verbose:
            print(f"SP2 | status={status} | obj={obj_val} | vars/cons={m2.num_cols}/{m2.num_rows} | time={t1-t0:.2f}s")
        assert m2.num_cols == n_x1 + n_x2_u*S2, "Conteo de variables inesperado en SP2"
        assert m2.num_rows == S2,               "Conteo de restricciones inesperado en SP2"

        return {"t": 2, "status": str(status), "objective": obj_val, "n_vars": m2.num_cols, "n_constrs": m2.num_rows, "model": m2}

    # ---------- t == 3 ----------
    elif t == 3:
        c34 = np.concatenate([c3, c4]).astype(float) if (n_x3+n_x4)>0 else np.array([], float)
        w3_u = np.concatenate([w3, w4]).astype(float)  if (n_x3+n_x4)>0 else np.array([], float)

        # U2 = U2_base (filas = s2), U34 = [ U3_base | tile(mu4) ] (filas = s3)
        U2  = U2_base
        U34 = np.concatenate([U3_base, tile4], axis=1) if c34.size>0 else np.zeros((n_smpl, 0), float)
        n_x3_u = c34.size
        S2 = S3 = n_smpl; p2 = p3 = 1.0 / n_smpl
        m3 = Model(sense=MAXIMIZE, solver_name=solver_name); m3.max_seconds = time_limit
        x1 = [m3.add_var(var_type=CONTINUOUS, lb=0.0, ub=1.0, name=f"x1_{j}") for j in range(n_x1)]
        x2 = {(s2, j): m3.add_var(var_type=CONTINUOUS, lb=0.0, ub=1.0, name=f"x2_{s2}_{j}") for s2 in range(S2) for j in range(n_x2)}
        x3 = {(s2, s3, k): m3.add_var(var_type=CONTINUOUS, lb=0.0, ub=1.0, name=f"x3_{s2}_{s3}_{k}") for s2 in range(S2) for s3 in range(S3) for k in range(n_x3_u)}

        obj = 0.0
        for j in range(n_x1): obj += c1[j] * x1[j]
        for s2 in range(S2):
            for j in range(n_x2):
                obj += (p2 * U2[s2, j]) * x2[(s2, j)]
        for s2 in range(S2):
            for s3 in range(S3):
                for k in range(n_x3_u):
                    obj += (p2 * p3 * U34[s3, k]) * x3[(s2, s3, k)]
        m3.objective = obj

        for s2 in range(S2):
            for s3 in range(S3):
                expr = 0.0
                for j in range(n_x1):    expr += w1[j]   * x1[j]
                for j in range(n_x2):    expr += w2[j]   * x2[(s2, j)]
                for k in range(n_x3_u):  expr += w3_u[k] * x3[(s2, s3, k)]
                m3 += (expr <= capacidad), f"cap_{s2}_{s3}"

        t0 = time.perf_counter(); status = m3.optimize(); t1 = time.perf_counter()
        solved = status in (OptimizationStatus.OPTIMAL, OptimizationStatus.FEASIBLE)
        obj_val = m3.objective_value if solved else None

        if verbose:
            print(f"SP3 | status={status} | obj={obj_val} | vars/cons={m3.num_cols}/{m3.num_rows} | time={t1-t0:.2f}s")
        assert m3.num_cols == n_x1 + n_x2*(S2) + n_x3_u*(S2*S3), "Conteo de variables inesperado en SP3"
        assert m3.num_rows == S2*S3,                              "Conteo de restricciones inesperado en SP3"

        return {"t": 3, "status": str(status), "objective": obj_val, "n_vars": m3.num_cols, "n_constrs": m3.num_rows, "model": m3}
    # ---------- t == 4 ----------
    else:
        U2, U3, U4 = U2_base, U3_base, U4_base
        S2 = S3 = S4 = n_smpl
        p2 = p3 = p4 = 1.0 / n_smpl

        m4 = Model(sense=MAXIMIZE, solver_name=solver_name); m4.max_seconds = time_limit
        x1 = [m4.add_var(var_type=CONTINUOUS, lb=0.0, ub=1.0, name=f"x1_{j}") for j in range(n_x1)]
        x2 = {(s2, j): m4.add_var(var_type=CONTINUOUS, lb=0.0, ub=1.0, name=f"x2_{s2}_{j}")
              for s2 in range(S2) for j in range(n_x2)}
        x3 = {(s2, s3, k): m4.add_var(var_type=CONTINUOUS, lb=0.0, ub=1.0, name=f"x3_{s2}_{s3}_{k}")
              for s2 in range(S2) for s3 in range(S3) for k in range(n_x3)}
        x4 = {(s2, s3, s4, h): m4.add_var(var_type=CONTINUOUS, lb=0.0, ub=1.0, name=f"x4_{s2}_{s3}_{s4}_{h}")
              for s2 in range(S2) for s3 in range(S3) for s4 in range(S4) for h in range(n_x4)}
        obj = 0.0
        for j in range(n_x1):
            obj += c1[j] * x1[j]
        for s2 in range(S2):
            for j in range(n_x2):
                obj += (p2 * U2[s2, j]) * x2[(s2, j)]
        for s2 in range(S2):
            for s3 in range(S3):
                for k in range(n_x3):
                    obj += (p2 * p3 * U3[s3, k]) * x3[(s2, s3, k)]
        for s2 in range(S2):
            for s3 in range(S3):
                for s4 in range(S4):
                    for h in range(n_x4):
                        obj += (p2 * p3 * p4 * U4[s4, h]) * x4[(s2, s3, s4, h)]
        m4.objective = obj
        for s2 in range(S2):
            for s3 in range(S3):
                for s4 in range(S4):
                    expr = 0.0
                    for j in range(n_x1): expr += w1[j] * x1[j]
                    for j in range(n_x2): expr += w2[j] * x2[(s2, j)]
                    for k in range(n_x3): expr += w3[k] * x3[(s2, s3, k)]
                    for h in range(n_x4): expr += w4[h] * x4[(s2, s3, s4, h)]
                    m4 += (expr <= capacidad), f"cap_{s2}_{s3}_{s4}"
        status = m4.optimize()
        solved  = status in (OptimizationStatus.OPTIMAL, OptimizationStatus.FEASIBLE)
        obj_val = m4.objective_value if solved else None
        if verbose:
            print(f"SP4 | status={status} | obj={obj_val} | vars/cons={m4.num_cols}/{m4.num_rows} | time={t1-t0:.2f}s")
        exp_vars = n_x1 + n_x2*S2 + n_x3*(S2*S3) + n_x4*(S2*S3*S4)
        exp_cons = S2*S3*S4
        assert m4.num_cols == exp_vars, f"Vars esperadas {exp_vars}, got {m4.num_cols}"
        assert m4.num_rows == exp_cons, f"Cons esperadas {exp_cons}, got {m4.num_rows}"
        return {"t": 4, "status": str(status), "objective": obj_val, "n_vars": m4.num_cols, "n_constrs": m4.num_rows, "model": m4}

filas = []
for pct_desviacion in (0.10, 0.30, 0.50):
    for n_target in sizes:
        df_sub = df_kplib[df_kplib["n"] == n_target].head(k)
        # print(f"\n=== n = {n_target} | {len(df_sub)} instancias ===")
        for idx, row in df_sub.iterrows():
            bar_c = np.array(row["Utilidades"], float)
            pesos = np.array(row["Pesos"], float)
            cap = float(row["Capacidad"])
            arc = row["Archivo"] if "Archivo" in df_kplib.columns else f"idx_{idx}"

            assert bar_c.size == pesos.size, f"Dim mismatch en {arc}"

            # print(f"\nInstancia: {arc} | n={bar_c.size} | cap={cap} | pct={pct_desviacion} | n_smpl={n_smpl} | seed={seed}")

            for t in (1, 2, 3, 4):
                t0 = time.perf_counter()
                sol = solve_sp(bar_c, pesos, cap, pct_desviacion, t=t, n_smpl=n_smpl, seed=seed, verbose=False)
                t1 = time.perf_counter()
                obj = float(sol["objective"]) if sol["objective"] is not None else np.nan
                dur = round(t1 - t0, 4)
                # print(f"  SP{t}: status={sol['status']:<20} | obj={obj:.4f} | vars={sol['n_vars']} | cons={sol['n_constrs']} | time={dur:.2f}s")

                filas.append({
                    "Archivo": arc,
                    "n": int(row["n"]),
                    "Capacidad": cap,
                    "%Desv": pct_desviacion,
                    "n_smpl": n_smpl,
                    "seed": seed,
                    "t": t,
                    "objective": obj,
                    "status": sol["status"],
                    "n_vars": sol["n_vars"],
                    "n_constrs": sol["n_constrs"],
                    "time_s": dur,
                })

# --- Resultados en DataFrame (formato largo) ---
df_res_sp = pd.DataFrame(filas)
df_res_sp["objective"] = df_res_sp["objective"].round(6)

print("\n=== Resumen (primeras filas) ===")
print(df_res_sp.head(10))

# --- Tabla ancha: objetivos SP1–SP4 por instancia ---
keys = ["Archivo", "n", "Capacidad", "%Desv"]
wide_obj_sp = (df_res_sp.pivot(index=keys, columns="t", values="objective")
            .rename(columns={1: "SP1", 2: "SP2", 3: "SP3", 4: "SP4"})
            .reset_index())

# --- Tabla ancha: tiempos SP1–SP4 por instancia ---
wide_time_sp = (df_res_sp.pivot(index=keys, columns="t", values="time_s")
             .rename(columns={1: "tiempo_SP1", 2: "tiempo_SP2", 3: "tiempo_SP3", 4: "tiempo_SP4"})
             .reset_index())

# --- Unimos objetivos + tiempos ---
wide_sp = pd.merge(wide_obj_sp, wide_time_sp, on=keys, how="inner")

print("\n=== Tabla ancha SP1–SP4 (primeras filas) ===")
print(wide_sp.head(10))






=== Resumen (primeras filas) ===
   Archivo   n  Capacidad  %Desv  n_smpl  seed  t     objective  \
0  s000.kp  50    14778.0    0.1       8   123  1  20909.186662   
1  s000.kp  50    14778.0    0.1       8   123  2  20909.186662   
2  s000.kp  50    14778.0    0.1       8   123  3  20916.748021   
3  s000.kp  50    14778.0    0.1       8   123  4  20931.908309   
4  s001.kp  50    13598.0    0.1       8   123  1  19775.549729   
5  s001.kp  50    13598.0    0.1       8   123  2  19775.549729   
6  s001.kp  50    13598.0    0.1       8   123  3  19775.549729   
7  s001.kp  50    13598.0    0.1       8   123  4  19775.549729   
8  s002.kp  50    13810.0    0.1       8   123  1  20610.498812   
9  s002.kp  50    13810.0    0.1       8   123  2  20610.498812   

                       status  n_vars  n_constrs  time_s  
0  OptimizationStatus.OPTIMAL      50          1  0.1075  
1  OptimizationStatus.OPTIMAL     316          8  0.0144  
2  OptimizationStatus.OPTIMAL    1772         64  0

In [7]:
wide_obj_sp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Archivo    90 non-null     object 
 1   n          90 non-null     int64  
 2   Capacidad  90 non-null     float64
 3   %Desv      90 non-null     float64
 4   SP1        90 non-null     float64
 5   SP2        90 non-null     float64
 6   SP3        90 non-null     float64
 7   SP4        90 non-null     float64
dtypes: float64(6), int64(1), object(1)
memory usage: 5.8+ KB


In [8]:
wide_obj_sp.to_excel("wide_obj_sp.xlsx")
wide_time_sp.to_excel("wide_time_sp.xlsx")
wide_sp.to_excel("wide_sp.xlsx")

from google.colab import files

files.download("wide_obj_sp.xlsx")
files.download("wide_time_sp.xlsx")
files.download("wide_sp.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## bertsimas model

In [9]:
# ================== Bertsimas–Sim (fraccional) con MIP + Excel (XlsxWriter) ==================
import sys, subprocess, time
import numpy as np
import pandas as pd
from mip import Model, CONTINUOUS, MAXIMIZE, CBC, OptimizationStatus

# # --- Descarga en Colab (si procede) ---
# try:
#     from google.colab import files
#     IN_COLAB = True
# except Exception:
#     IN_COLAB = False

# # --- Asegurar XlsxWriter para Excel ---
# try:
#     import xlsxwriter  # noqa: F401
#     EXCEL_ENGINE = "xlsxwriter"
# except ModuleNotFoundError:
#     print("[Info] Instalando XlsxWriter…")
#     subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "XlsxWriter"])
#     import xlsxwriter  # noqa: F401
#     EXCEL_ENGINE = "xlsxwriter"


def solve_nominal_fractional_mip(c_bar, pesos, capacidad, time_limit=3600, solver_name=CBC):
    """Max sum c_bar x  s.a. sum w x <= cap, 0<=x<=1 (fraccional) con MIP."""
    c_bar = np.asarray(c_bar, float)
    pesos = np.asarray(pesos, float)
    n = c_bar.size

    m = Model(sense=MAXIMIZE, solver_name=solver_name)
    m.max_seconds = time_limit
    x = [m.add_var(var_type=CONTINUOUS, lb=0.0, ub=1.0, name=f"x_{j}") for j in range(n)]

    m.objective = sum(c_bar[j]*x[j] for j in range(n))
    m += (sum(pesos[j]*x[j] for j in range(n)) <= capacidad), "cap"

    t0 = time.perf_counter(); status = m.optimize(); t1 = time.perf_counter()
    obj = m.objective_value if status in (OptimizationStatus.OPTIMAL, OptimizationStatus.FEASIBLE) else np.nan
    return float(obj), status, t1-t0


def solve_knapsack_bs_mip(c_bar, c_hat, pesos, capacidad, gamma, time_limit=3600, solver_name=CBC):
    """
    Bertsimas–Sim robusto (fraccional, x in [0,1]) con MIP (CBC):
      max  sum c_bar_j x_j  - gamma*z  - sum p_j
      s.a. d_j x_j - z - p_j <= 0  (∀j)
           sum w_j x_j <= capacidad
           x ∈ [0,1], z>=0, p>=0
    Devuelve además eval_cx = sum c_bar_j x_j (evaluación nominal de la solución x*).
    """
    c_bar = np.asarray(c_bar, float)
    c_hat = np.asarray(c_hat, float)
    pesos = np.asarray(pesos, float)
    n = c_bar.size
    assert c_hat.size == n and pesos.size == n, "Dimensiones incoherentes"

    m = Model(sense=MAXIMIZE, solver_name=solver_name)
    m.max_seconds = time_limit

    # Variables
    x = [m.add_var(var_type=CONTINUOUS, lb=0.0, ub=1.0, name=f"x_{j}") for j in range(n)]
    z = m.add_var(var_type=CONTINUOUS, lb=0.0, name="z")
    p = [m.add_var(var_type=CONTINUOUS, lb=0.0, name=f"p_{j}") for j in range(n)]

    # Objetivo robusto
    m.objective = sum(c_bar[j]*x[j] for j in range(n)) - gamma*z - sum(p[j] for j in range(n))

    # Capacidad
    m += (sum(pesos[j]*x[j] for j in range(n)) <= capacidad), "cap"

    # Robustez: d_j x_j - z - p_j <= 0
    for j in range(n):
        m += (c_hat[j]*x[j] - z - p[j] <= 0), f"rob_{j}"

    t0 = time.perf_counter()
    status = m.optimize()
    t1 = time.perf_counter()
    ok = status in (OptimizationStatus.OPTIMAL, OptimizationStatus.FEASIBLE)
    obj = float(m.objective_value) if ok else np.nan

    # Extra: evaluación nominal de la solución x* (sin penalizaciones)
    if ok:
        sol_x = np.array([var.x for var in x], dtype=float)
        eval_cx = float(np.dot(c_bar, sol_x))
        sol_z = float(z.x)
        sol_p = np.array([var.x for var in p], dtype=float)
    else:
        sol_x = None
        eval_cx = np.nan
        sol_z = np.nan
        sol_p = None

    return {
        "ok": ok,
        "status": str(status),
        "objective": obj,         # robusto: c·x - gamma z - sum p
        "eval_cx": eval_cx,       # nominal: c·x
        "x": sol_x,
        "z": sol_z,
        "p": sol_p,
        "time_s": round(t1 - t0, 6),
        "n_vars": m.num_cols,
        "n_constrs": m.num_rows,
        "model": m,
    }


def total_solver_bs_mip(
    df_kplib,
    pct_desviacion_list=(0.10, 0.30, 0.50),
    gamma_pcts=(0.00, 0.05, 0.10, 0.25, 0.50, 0.75, 1.00),
    sizes=(50, 100, 200, 500),
    k=20,
    time_limit=3600,
    check_nominal=True,
):
    filas = []

    for pct_desviacion in pct_desviacion_list:
        for n_target in sizes:
            df_sub = df_kplib[df_kplib["n"] == n_target].head(k)
            print(f"\n=== n = {n_target} | {len(df_sub)} instancias | pct_desv={pct_desviacion} ===")
            for idx, row in df_sub.iterrows():
                cbar = np.array(row["Utilidades"], float)
                w    = np.array(row["Pesos"], float)
                cap  = float(row["Capacidad"])
                arc  = row["Archivo"] if "Archivo" in df_kplib.columns else f"idx_{idx}"
                n    = cbar.size
                assert w.size == n, f"Dim mismatch en {arc}"

                # d_j = pct * |c̄_j|
                chat = np.abs(pct_desviacion * cbar)

                # Chequeo nominal (Γ=0) vs LP nominal fraccional (con MIP también)
                if check_nominal:
                    f_nom, st_nom, _ = solve_nominal_fractional_mip(cbar, w, cap, time_limit=time_limit)
                else:
                    f_nom = np.nan

                for pct in gamma_pcts:
                    gamma = int(round(pct * n))
                    sol = solve_knapsack_bs_mip(cbar, chat, w, cap, gamma, time_limit=time_limit)

                    if sol["ok"]:
                        if check_nominal and gamma == 0 and np.isfinite(f_nom):
                            diff = abs(sol["objective"] - f_nom)
                            if diff > 1e-6:
                                print(f"[Aviso] {arc} | Γ=0: nominal={f_nom:.6f}, robust={sol['objective']:.6f} (ambos fraccionales)")

                        filas.append({
                            "Archivo": arc,
                            "n": int(row["n"]),
                            "Capacidad": cap,
                            "%Desv": pct_desviacion,
                            "Gamma": gamma,
                            "%Gamma": pct,
                            "Valor objetivo": sol["objective"],   # robusto
                            "Eval c·x": sol["eval_cx"],            # nominal de la solución
                            "z": sol["z"],
                            "Tiempo (s)": sol["time_s"],
                            "Status": sol["status"],
                            "n_vars": sol["n_vars"],
                            "n_constrs": sol["n_constrs"],
                        })
                    else:
                        print(f"  {arc} | Γ={gamma}: fallo -> {sol['status']}")

    # ---- DataFrame largo ----
    df_long = pd.DataFrame(filas)
    if not df_long.empty:
        df_long[["Valor objetivo", "Eval c·x", "z", "Tiempo (s)"]] = df_long[["Valor objetivo", "Eval c·x", "z", "Tiempo (s)"]].round(6)

    print("\n=== Resumen (primeras filas) ===")
    print(df_long.head(10))
    # ---- Tabla wide: objetivos, evaluación c·x y tiempos por Γ ----
    keys = ["Archivo", "n", "Capacidad", "%Desv"]

    def rename_cols(df, prefix):
        ren = {}
        for col in df.columns:
            if isinstance(col, float):  # columnas de %Gamma
                ren[col] = f"{prefix}_Gamma{int(round(col*100))}%"
        return df.rename(columns=ren)

    # f(Γ)
    wide_obj = (
        df_long.pivot(index=keys, columns="%Gamma", values="Valor objetivo")
              .sort_index(axis=1)
              .reset_index()
    )
    wide_obj = rename_cols(wide_obj, "f")

    # c·x(Γ)
    wide_cx = (
        df_long.pivot(index=keys, columns="%Gamma", values="Eval c·x")
              .sort_index(axis=1)
              .reset_index()
    )
    wide_cx = rename_cols(wide_cx, "cx")

    # t(Γ)
    wide_time = (
        df_long.pivot(index=keys, columns="%Gamma", values="Tiempo (s)")
              .sort_index(axis=1)
              .reset_index()
    )
    wide_time = rename_cols(wide_time, "tiempo")

    # Unimos
    df_wide = wide_obj.merge(wide_cx, on=keys, how="inner").merge(wide_time, on=keys, how="inner")

    # print("\n=== Wide (primeras filas) ===")
    # print(df_wide.head(10))

    # # ---- Guardar Excel con XlsxWriter ----
    # out_name = "BertsimasSim_MIP.xlsx"
    # with pd.ExcelWriter(out_name, engine=EXCEL_ENGINE) as writer:
    #     df_long.to_excel(writer, sheet_name="long", index=False)
    #     df_wide.to_excel(writer, sheet_name="wide", index=False)

    # if IN_COLAB:
    #     files.download(out_name)
    # else:
    #     print(f"[Info] Archivo guardado: {out_name}")

    return df_long, df_wide, wide_time, wide_obj, wide_cx


# ================== EJECUCIÓN (ejemplo) ==================
long_bs, wide_bs, wide_time_bs, wide_obj_bs, wide_cx_bs = total_solver_bs_mip(
    df_kplib,
    pct_desviacion_list=(0.10, 0.30, 0.50),
    gamma_pcts=(0.00, 0.05, 0.10, 0.25, 0.50, 0.75, 1.00),
    sizes=(50, 100, 200, 500),
    k=20,
    time_limit=3600,
    check_nominal=True,
)





=== n = 50 | 15 instancias | pct_desv=0.1 ===

=== n = 100 | 15 instancias | pct_desv=0.1 ===

=== n = 200 | 15 instancias | pct_desv=0.1 ===

=== n = 500 | 15 instancias | pct_desv=0.1 ===

=== n = 50 | 15 instancias | pct_desv=0.3 ===

=== n = 100 | 15 instancias | pct_desv=0.3 ===

=== n = 200 | 15 instancias | pct_desv=0.3 ===

=== n = 500 | 15 instancias | pct_desv=0.3 ===

=== n = 50 | 15 instancias | pct_desv=0.5 ===

=== n = 100 | 15 instancias | pct_desv=0.5 ===

=== n = 200 | 15 instancias | pct_desv=0.5 ===

=== n = 500 | 15 instancias | pct_desv=0.5 ===

=== Resumen (primeras filas) ===
   Archivo   n  Capacidad  %Desv  Gamma  %Gamma  Valor objetivo      Eval c·x  \
0  s000.kp  50    14778.0    0.1      0    0.00    21047.483259  21047.483259   
1  s000.kp  50    14778.0    0.1      2    0.05    20852.383259  21047.483259   
2  s000.kp  50    14778.0    0.1      5    0.10    20573.437423  21047.137423   
3  s000.kp  50    14778.0    0.1     12    0.25    19969.468937  2103

In [10]:
wide_obj_bs.to_excel("wide_obj_bs.xlsx")
wide_time_bs.to_excel("wide_time_bs.xlsx")
wide_bs.to_excel("wide_bs.xlsx")
files.download("wide_obj_bs.xlsx")
files.download("wide_time_bs.xlsx")
files.download("wide_bs.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# estudio valor objetivo

In [11]:
wide_obj_bs.info()
wide_obj_sp.info()

import numpy as np
import pandas as pd

# ========= 1) Unir ambos wide (objetivo) por claves =========
keys = ["Archivo", "n", "Capacidad", "%Desv"]
wide_obj = wide_obj_bs.merge(wide_obj_sp, on=keys, how="inner")

# ========= 2) Seleccionar columnas de objetivo y pasar a largo =========
cols_sp_obj = ["SP1", "SP2", "SP3", "SP4"]
cols_bs_obj = [c for c in wide_obj.columns if c.startswith("f_Gamma")]
cols_obj = cols_sp_obj + cols_bs_obj

long_obj = wide_obj.melt(
    id_vars=["Archivo", "n", "%Desv"],
    value_vars=cols_obj,
    var_name="Metodo",
    value_name="Valor"
)

# Normalizamos nombres de métodos Gamma (opcional estético)
long_obj["Metodo"] = long_obj["Metodo"].str.replace("^f_", "", regex=True)  # f_GammaX% -> GammaX%

# ========= 3) Promedio sobre %Desv por (Archivo, n, Metodo) =========
long_obj_avg = (
    long_obj
    .groupby(["Archivo", "n", "Metodo"], as_index=False)["Valor"]
    .mean()
)

# ========= 4) Estadísticos por (n, Metodo) =========
df_stats_obj = (
    long_obj_avg
    .groupby(["n", "Metodo"])["Valor"]
    .agg(
        Mediana="median",
        Q1=lambda x: np.percentile(x, 25),
        Q3=lambda x: np.percentile(x, 75),
        Media="mean",
        **{"Desv. típica": "std"}
    )
    .reset_index()
)

# ========= 5) Funciones auxiliares de tablas (igual que en tiempos) =========
def tabla_mediana(df, escala=1.0, dec=4, con_intervalo=False):
    df2 = df.copy()
    if con_intervalo:
        df2["Mediana [Q1, Q3]"] = df2.apply(
            lambda r: f"{r['Mediana']*escala:.{dec}f} [{r['Q1']*escala:.{dec}f}, {r['Q3']*escala:.{dec}f}]",
            axis=1
        )
        out = df2.pivot(index="n", columns="Metodo", values="Mediana [Q1, Q3]").reset_index()
    else:
        df2["Mediana"] = (df2["Mediana"]*escala).round(dec)
        out = df2.pivot(index="n", columns="Metodo", values="Mediana").reset_index()
    return out

def tabla_media(df, escala=1.0, dec=4, con_desv=False):
    df2 = df.copy()
    if con_desv:
        df2["Media ± Desv. típica"] = df2.apply(
            lambda r: f"{r['Media']*escala:.{dec}f} ± {r['Desv. típica']*escala:.{dec}f}",
            axis=1
        )
        out = df2.pivot(index="n", columns="Metodo", values="Media ± Desv. típica").reset_index()
    else:
        df2["Media"] = (df2["Media"]*escala).round(dec)
        out = df2.pivot(index="n", columns="Metodo", values="Media").reset_index()
    return out

# ========= 6) Generar tablas (sin escalado; ajusta escala/decimales si lo necesitas) =========
# (a) Mediana sola
tabla_obj_mediana = tabla_mediana(df_stats_obj, escala=1.0, dec=4, con_intervalo=False)

# (b) Media sola
tabla_obj_media = tabla_media(df_stats_obj, escala=1.0, dec=4, con_desv=False)

# (c) Mediana [Q1,Q3]
tabla_obj_mediana_iqr = tabla_mediana(df_stats_obj, escala=1.0, dec=4, con_intervalo=True)

# (d) Media ± Desv. típica
tabla_obj_media_desv = tabla_media(df_stats_obj, escala=1.0, dec=4, con_desv=True)

# ========= 7) (Opcional) ordenar columnas: SP1..SP4 y luego Gammas ascendentes =========
orden_cols = (["n"] + cols_sp_obj + sorted([c.replace("f_", "") for c in cols_bs_obj]))
for df_out in (tabla_obj_mediana, tabla_obj_media, tabla_obj_mediana_iqr, tabla_obj_media_desv):
    existing = [c for c in orden_cols if c in df_out.columns]
    df_out.sort_index(axis=1, inplace=True)  # orden alfabético por si acaso
    df_out = df_out.reindex(columns=(["n"] + [c for c in orden_cols[1:] if c in df_out.columns]))

display(tabla_obj_mediana)
display(tabla_obj_media)
display(tabla_obj_mediana_iqr)
display(tabla_obj_media_desv)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Archivo      180 non-null    object 
 1   n            180 non-null    int64  
 2   Capacidad    180 non-null    float64
 3   %Desv        180 non-null    float64
 4   f_Gamma0%    180 non-null    float64
 5   f_Gamma5%    180 non-null    float64
 6   f_Gamma10%   180 non-null    float64
 7   f_Gamma25%   180 non-null    float64
 8   f_Gamma50%   180 non-null    float64
 9   f_Gamma75%   180 non-null    float64
 10  f_Gamma100%  180 non-null    float64
dtypes: float64(9), int64(1), object(1)
memory usage: 15.6+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Archivo    90 non-null     object 
 1   n          90 non-null     int64  
 2   Capacidad  90 non-n

Metodo,Gamma0%,Gamma10%,Gamma100%,Gamma25%,Gamma5%,Gamma50%,Gamma75%,SP1,SP2,SP3,SP4,n
0,19845.2576,18458.2977,13891.6803,16946.7870,19267.0344,14767.7873,13891.6803,19822.5752,19893.2985,19991.8537,20041.6202,50
1,39747.3263,36915.7233,27823.1284,33406.0296,38285.7263,29382.5978,27848.7982,39745.7262,39853.3642,39905.7014,39971.3328,100


Metodo,Gamma0%,Gamma10%,Gamma100%,Gamma25%,Gamma5%,Gamma50%,Gamma75%,SP1,SP2,SP3,SP4,n
0,19797.4583,18396.6334,13858.2208,16739.8254,19216.3013,14602.4644,13860.9102,19637.7598,19686.5331,19734.5873,19765.8133,50
1,40334.2686,37493.8994,28233.9880,33868.1012,38872.2071,29707.9756,28242.7194,40521.8602,40597.4457,40665.4550,40725.9601,100


Metodo,Gamma0%,Gamma10%,Gamma100%,Gamma25%,Gamma5%,Gamma50%,Gamma75%,SP1,SP2,SP3,SP4,n
0,"19845.2576 [19288.5206, 20539.1369]","18458.2977 [17867.6685, 19073.8771]","13891.6803 [13501.9644, 14377.3958]","16946.7870 [16218.3537, 17314.1915]","19267.0344 [18697.1222, 19941.7370]","14767.7873 [14138.1530, 15222.1990]","13891.6803 [13504.1074, 14394.4458]","19822.5752 [19015.9668, 20423.9865]","19893.2985 [19071.3858, 20467.5630]","19991.8537 [19101.8005, 20495.8093]","20041.6202 [19109.9163, 20535.6906]",50
1,"39747.3263 [37899.3252, 41924.3977]","36915.7233 [35067.3079, 39077.8677]","27823.1284 [26529.5276, 29347.0784]","33406.0296 [31603.1526, 35302.2467]","38285.7263 [36435.6827, 40461.0020]","29382.5978 [27706.5431, 30738.5660]","27848.7982 [26529.8263, 29347.0784]","39745.7262 [38154.3413, 42068.6754]","39853.3642 [38181.2307, 42126.8949]","39905.7014 [38238.7331, 42191.7363]","39971.3328 [38268.3519, 42263.5069]",100


Metodo,Gamma0%,Gamma10%,Gamma100%,Gamma25%,Gamma5%,Gamma50%,Gamma75%,SP1,SP2,SP3,SP4,n
0,19797.4583 ± 1185.3250,18396.6334 ± 1168.5596,13858.2208 ± 829.7275,16739.8254 ± 1085.6343,19216.3013 ± 1181.6493,14602.4644 ± 951.7546,13860.9102 ± 831.0804,19637.7598 ± 1208.5093,19686.5331 ± 1197.8991,19734.5873 ± 1219.7667,19765.8133 ± 1213.6830,50
1,40334.2686 ± 3091.3613,37493.8994 ± 3080.4617,28233.9880 ± 2163.9529,33868.1012 ± 2910.7207,38872.2071 ± 3089.1480,29707.9756 ± 2569.3218,28242.7194 ± 2176.2971,40521.8602 ± 3172.4450,40597.4457 ± 3167.7955,40665.4550 ± 3174.4672,40725.9601 ± 3185.7236,100


In [12]:
import numpy as np
import pandas as pd

# Claves comunes
keys = ["Archivo", "n", "Capacidad", "%Desv"]

# Unimos objetivos SP y RO (Bertsimas–Sim)
wide_obj_all = wide_obj_bs.merge(wide_obj_sp, on=keys, how="inner")

# Columnas de métodos
cols_gamma = [c for c in wide_obj_all.columns if c.startswith("f_Gamma")]
cols_sp    = ["SP1", "SP2", "SP3", "SP4"]

# ========= 1) Incrementos % respecto a SP1 =========
baseline_sp = wide_obj_all["SP1"].astype(float)

# Métodos a comparar vs SP1: SP2..SP4 + todos los Gamma
cols_vs_sp = ["SP2", "SP3", "SP4"] + cols_gamma

def pct_inc(col_vals, base):
    col_vals = col_vals.astype(float)
    # Evitar división por 0: si base==0 -> NaN
    return np.where(base != 0.0, (col_vals - base) / base, np.nan)

wide_obj_vs_sp = wide_obj_all[keys].copy()
# SP1 como 0%
wide_obj_vs_sp["SP1"] = 0.0
# Resto respecto a SP1
for c in cols_vs_sp:
    wide_obj_vs_sp[c] = pct_inc(wide_obj_all[c], baseline_sp)

# (Opcional) orden de columnas
wide_obj_vs_sp = wide_obj_vs_sp[keys + cols_sp + cols_gamma]

# ========= 2) Incrementos % respecto a nominal (Gamma = 0%) =========
baseline_nom = wide_obj_all["f_Gamma0%"].astype(float)

# Métodos a comparar vs Γ=0%: SP1..SP4 + Gamma5..Gamma100 (dejamos Gamma0 como 0)
cols_vs_nom = cols_sp + [c for c in cols_gamma if c != "f_Gamma0%"]

wide_obj_vs_nom = wide_obj_all[keys].copy()
# Gamma0% como 0%
wide_obj_vs_nom["f_Gamma0%"] = 0.0
# Resto respecto a Gamma0%
for c in cols_vs_nom:
    wide_obj_vs_nom[c] = pct_inc(wide_obj_all[c], baseline_nom)

# (Opcional) orden de columnas: claves + SP + Gammas (Gamma0 primero)
wide_obj_vs_nom = wide_obj_vs_nom[keys + cols_sp + ["f_Gamma0%"] + [c for c in cols_gamma if c != "f_Gamma0%"]]

# (Opcional) redondeo
wide_obj_vs_sp  = wide_obj_vs_sp.round(6)
wide_obj_vs_nom = wide_obj_vs_nom.round(6)

# Vistas rápidas
print("=== Incrementos % respecto a SP1 ===")
display(wide_obj_vs_sp.head())
print("\n=== Incrementos % respecto a Γ=0% (nominal) ===")
display(wide_obj_vs_nom.head())


=== Incrementos % respecto a SP1 ===


,Archivo,n,Capacidad,%Desv,SP1,SP2,SP3,SP4,f_Gamma0%,f_Gamma5%,f_Gamma10%,f_Gamma25%,f_Gamma50%,f_Gamma75%,f_Gamma100%
0,s000.kp,50,14778.0,0.1,0.0,0.000000,0.000362,0.001087,0.006614,-0.002717,-0.016057,-0.044943,-0.085225,-0.094047,-0.094047
1,s000.kp,50,14778.0,0.3,0.0,0.000548,0.005282,0.008077,0.014892,-0.013331,-0.053316,-0.137346,-0.252880,-0.289576,-0.289576
2,s000.kp,50,14778.0,0.5,0.0,0.003169,0.011685,0.016401,0.016760,-0.030364,-0.096597,-0.232421,-0.412329,-0.491620,-0.491620
3,s000.kp,100,22545.0,0.1,0.0,0.000098,0.000296,0.000422,-0.003345,-0.013782,-0.023726,-0.051736,-0.087369,-0.103010,-0.103010
4,s000.kp,100,22545.0,0.3,0.0,0.000457,0.001552,0.003022,-0.009979,-0.041083,-0.070715,-0.154186,-0.258020,-0.306985,-0.306985



=== Incrementos % respecto a Γ=0% (nominal) ===


,Archivo,n,Capacidad,%Desv,SP1,SP2,SP3,SP4,f_Gamma0%,f_Gamma5%,f_Gamma10%,f_Gamma25%,f_Gamma50%,f_Gamma75%,f_Gamma100%
0,s000.kp,50,14778.0,0.1,-0.006571,-0.006571,-0.006211,-0.005491,0.0,-0.009270,-0.022523,-0.051218,-0.091236,-0.1,-0.1
1,s000.kp,50,14778.0,0.3,-0.014673,-0.014133,-0.009469,-0.006714,0.0,-0.027809,-0.067207,-0.150003,-0.263843,-0.3,-0.3
2,s000.kp,50,14778.0,0.5,-0.016484,-0.013367,-0.004992,-0.000354,0.0,-0.046348,-0.111489,-0.245074,-0.422017,-0.5,-0.5
3,s000.kp,100,22545.0,0.1,0.003356,0.003455,0.003653,0.003779,0.0,-0.010473,-0.020450,-0.048554,-0.084306,-0.1,-0.1
4,s000.kp,100,22545.0,0.3,0.010079,0.010541,0.011647,0.013132,0.0,-0.031418,-0.061349,-0.145661,-0.250542,-0.3,-0.3


In [13]:
wide_obj_vs_sp.to_excel("wide_obj_vs_sp.xlsx")
wide_obj_vs_nom.to_excel("wide_obj_vs_nom.xlsx")

files.download("wide_obj_vs_sp.xlsx")
files.download("wide_obj_vs_nom.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
df_nom_agg = (
    wide_obj_vs_nom
    .groupby(["n"], as_index=False)      # Agrupa por n
    .mean(numeric_only=True)            # Calcula la media de TODAS las columnas numéricas
)
df_nom_agg.to_excel("df_nom_agg_SOLO_NUM_ITEMS.xlsx")
files.download("df_nom_agg_SOLO_NUM_ITEMS.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
df_nom_agg_STD = (
    wide_obj_vs_nom
    .groupby(["n"], as_index=False)      # Agrupa por n
    .std(numeric_only=True)             # Calcula la desviación típica de TODAS las columnas numéricas
)

df_nom_agg_STD.to_excel("df_nom_agg_SOLO_NUM_ITEMS_STD.xlsx")
files.download("df_nom_agg_SOLO_NUM_ITEMS_STD.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
df_agg_incertidumbre = (
    wide_obj_vs_nom
    .groupby(["n", "%Desv"], as_index=False)
    .mean(numeric_only=True)
)

df_agg_incertidumbre_STD = (
    wide_obj_vs_nom
    .groupby(["n", "%Desv"], as_index=False)
    .std(numeric_only=True)
)

df_agg_incertidumbre.to_excel("df_agg_incertidumbre.xlsx")
df_agg_incertidumbre_STD.to_excel("df_agg_incertidumbre_STD.xlsx")
files.download("df_agg_incertidumbre.xlsx")
files.download("df_agg_incertidumbre_STD.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# estudio de tiempo de computación

In [9]:
import numpy as np
import pandas as pd


wide_time = wide_time_bs.merge(
    wide_time_sp, on=["Archivo", "n", "Capacidad", "%Desv"],
    how="inner"
)
# --- Paso 1: Unir los dataframes (ya lo tienes) ---
# wide_time = wide_time_bs.merge(wide_time_sp, on=["Archivo", "n", "Capacidad", "%Desv"], how="inner")

# --- Paso 2: Seleccionar columnas de tiempo (mantenemos Archivo, n, %Desv) ---
cols_sp = ["tiempo_SP1", "tiempo_SP2", "tiempo_SP3", "tiempo_SP4"]
cols_bs = [c for c in wide_time.columns if c.startswith("tiempo_Gamma")]
cols_tiempo = cols_sp + cols_bs
base = wide_time[["Archivo", "n", "%Desv"] + cols_tiempo].copy()

# --- Paso 3: Formato largo conservando Archivo, n y %Desv ---
long_time = base.melt(
    id_vars=["Archivo", "n", "%Desv"],
    value_vars=cols_tiempo,
    var_name="Metodo",
    value_name="Tiempo"
)

# --- Paso 4: Promedio sobre %Desv por (Archivo, n, Metodo) ---
# (si hay varias filas para el mismo Archivo y n con %Desv=0.10/0.30/0.50, se promedia)
long_time_avg = (
    long_time
    .groupby(["Archivo", "n", "Metodo"], as_index=False)["Tiempo"]
    .mean()
)

# --- Paso 5: Estadísticos por (n, Metodo) usando las observaciones (Archivo) agregadas ---
def resumen(serie):
    med = serie.median()
    q1, q3 = np.percentile(serie, [25, 75])
    iqr = q3 - q1
    mean = serie.mean()
    std = serie.std(ddof=1)
    return pd.Series({
        "Median": med,
        "Median - IQR/2": med - iqr/2,
        "Median + IQR/2": med + iqr/2,
        "Mean": mean,
        "Mean - Std/2": mean - (std/2 if np.isfinite(std) else 0.0),
        "Mean + Std/2": mean + (std/2 if np.isfinite(std) else 0.0)
    })

df_stats_time = (
    long_time_avg
    .groupby(["n", "Metodo"])["Tiempo"]
    .apply(resumen)
    .reset_index()
)

# (Opcional) ordenar métodos: SP1..SP4 y luego Gamma ascendente
orden = ["tiempo_SP1","tiempo_SP2","tiempo_SP3","tiempo_SP4"] + sorted([c for c in cols_bs])
df_stats_time["Metodo"] = pd.Categorical(df_stats_time["Metodo"], categories=orden, ordered=True)
df_stats_time = df_stats_time.sort_values(["n","Metodo"]).reset_index(drop=True)

print(df_stats_time)


       n           Metodo         level_2    Tiempo
0     50       tiempo_SP1          Median  0.005133
1     50       tiempo_SP1  Median - IQR/2  0.004317
2     50       tiempo_SP1  Median + IQR/2  0.005950
3     50       tiempo_SP1            Mean  0.008462
4     50       tiempo_SP1    Mean - Std/2  0.001963
..   ...              ...             ...       ...
259  500  tiempo_Gamma75%  Median - IQR/2  0.005643
260  500  tiempo_Gamma75%  Median + IQR/2  0.009243
261  500  tiempo_Gamma75%            Mean  0.008102
262  500  tiempo_Gamma75%    Mean - Std/2  0.006870
263  500  tiempo_Gamma75%    Mean + Std/2  0.009334

[264 rows x 4 columns]


In [10]:
import numpy as np
import pandas as pd

# --- 1) Unir (asumo que ya tienes wide_time_bs y wide_time_sp) ---
wide_time = wide_time_bs.merge(
    wide_time_sp, on=["Archivo", "n", "Capacidad", "%Desv"], how="inner"
)

# --- 2) Seleccionar columnas de tiempo y pasar a largo ---
cols_sp = ["tiempo_SP1", "tiempo_SP2", "tiempo_SP3", "tiempo_SP4"]
cols_bs = [c for c in wide_time.columns if c.startswith("tiempo_Gamma")]
cols_tiempo = cols_sp + cols_bs

long_time = wide_time.melt(
    id_vars=["Archivo", "n", "%Desv"],
    value_vars=cols_tiempo,
    var_name="Metodo",
    value_name="Tiempo"
)

# --- 3) Promediar sobre %Desv por (Archivo, n, Metodo) ---
long_time_avg = (
    long_time
    .groupby(["Archivo", "n", "Metodo"], as_index=False)["Tiempo"]
    .mean()
)

# --- 4) Estadísticos por (n, Metodo) ---
df_stats = (
    long_time_avg
    .groupby(["n", "Metodo"])["Tiempo"]
    .agg(
        Mediana="median",
        Q1=lambda x: np.percentile(x, 25),
        Q3=lambda x: np.percentile(x, 75),
        Media="mean",
        **{"Desv. típica": "std"}
    )
    .reset_index()
)

# --- Funciones auxiliares ---
def tabla_mediana(df, escala=1.0, dec=4, con_intervalo=False):
    df2 = df.copy()
    if con_intervalo:
        df2["Mediana [Q1, Q3]"] = df2.apply(
            lambda r: f"{r['Mediana']*escala:.{dec}f} [{r['Q1']*escala:.{dec}f}, {r['Q3']*escala:.{dec}f}]",
            axis=1
        )
        out = df2.pivot(index="n", columns="Metodo", values="Mediana [Q1, Q3]").reset_index()
    else:
        df2["Mediana"] = (df2["Mediana"]*escala).round(dec)
        out = df2.pivot(index="n", columns="Metodo", values="Mediana").reset_index()
    return out

def tabla_media(df, escala=1.0, dec=4, con_desv=False):
    df2 = df.copy()
    if con_desv:
        df2["Media ± Desv. típica"] = df2.apply(
            lambda r: f"{r['Media']*escala:.{dec}f} ± {r['Desv. típica']*escala:.{dec}f}",
            axis=1
        )
        out = df2.pivot(index="n", columns="Metodo", values="Media ± Desv. típica").reset_index()
    else:
        df2["Media"] = (df2["Media"]*escala).round(dec)
        out = df2.pivot(index="n", columns="Metodo", values="Media").reset_index()
    return out

# --- 5) Generar tablas ---
# (a) Mediana sola en ms
tabla_mediana_ms = tabla_mediana(df_stats, escala=1e3, dec=1, con_intervalo=False)

# (b) Media sola en segundos
tabla_media_s = tabla_media(df_stats, escala=1.0, dec=4, con_desv=False)

# (c) Mediana [Q1,Q3] en ms
tabla_mediana_iqr_ms = tabla_mediana(df_stats, escala=1e3, dec=1, con_intervalo=True)

# (d) Media ± Desv. típica en ms
tabla_media_desv_ms = tabla_media(df_stats, escala=1e3, dec=1, con_desv=True)

# (e) Media ± Desv. típica en segundos
tabla_media_desv_s = tabla_media(df_stats, escala=1.0, dec=4, con_desv=True)

# --- 6) Mostrar ---
display(tabla_mediana_ms)
display(tabla_media_s)
display(tabla_mediana_iqr_ms)
display(tabla_media_desv_ms)
display(tabla_media_desv_s)



Metodo,n,tiempo_Gamma0%,tiempo_Gamma10%,tiempo_Gamma100%,tiempo_Gamma25%,tiempo_Gamma5%,tiempo_Gamma50%,tiempo_Gamma75%,tiempo_SP1,tiempo_SP2,tiempo_SP3,tiempo_SP4
0,50,0.7,0.9,1.1,1.1,1.0,1.2,1.2,5.1,24.8,184.4,1454.1
1,100,2.4,1.9,2.3,2.2,1.9,2.3,2.4,7.1,38.9,330.0,3458.3
2,200,3.0,3.0,3.7,4.1,2.8,4.2,3.7,11.6,77.1,736.4,9651.8
3,500,5.1,5.8,7.8,8.4,5.1,10.9,7.4,28.4,228.1,2893.3,50731.2


Metodo,n,tiempo_Gamma0%,tiempo_Gamma10%,tiempo_Gamma100%,tiempo_Gamma25%,tiempo_Gamma5%,tiempo_Gamma50%,tiempo_Gamma75%,tiempo_SP1,tiempo_SP2,tiempo_SP3,tiempo_SP4
0,50,0.0009,0.0009,0.0011,0.0011,0.0011,0.0013,0.0012,0.0085,0.0264,0.1984,1.4806
1,100,0.0031,0.0026,0.0026,0.0034,0.0020,0.0025,0.0029,0.0076,0.0413,0.3580,3.3507
2,200,0.0034,0.0030,0.0040,0.0044,0.0031,0.0044,0.0036,0.0129,0.0867,0.8129,9.6013
3,500,0.0052,0.0063,0.0078,0.0093,0.0055,0.0112,0.0081,0.0290,0.2357,3.0790,50.8823


Metodo,n,tiempo_Gamma0%,tiempo_Gamma10%,tiempo_Gamma100%,tiempo_Gamma25%,tiempo_Gamma5%,tiempo_Gamma50%,tiempo_Gamma75%,tiempo_SP1,tiempo_SP2,tiempo_SP3,tiempo_SP4
0,50,"0.7 [0.7, 0.9]","0.9 [0.8, 1.0]","1.1 [0.9, 1.3]","1.1 [0.9, 1.3]","1.0 [0.8, 1.3]","1.2 [1.0, 1.5]","1.2 [1.0, 1.4]","5.1 [4.2, 5.9]","24.8 [20.0, 30.9]","184.4 [162.6, 238.8]","1454.1 [1156.6, 1641.3]"
1,100,"2.4 [2.1, 3.8]","1.9 [1.7, 2.5]","2.3 [2.0, 2.7]","2.2 [2.0, 4.6]","1.9 [1.6, 2.0]","2.3 [2.0, 2.4]","2.4 [2.0, 3.7]","7.1 [6.8, 8.2]","38.9 [37.0, 44.7]","330.0 [314.3, 390.5]","3458.3 [3028.4, 3532.3]"
2,200,"3.0 [2.6, 3.6]","3.0 [2.8, 3.1]","3.7 [3.4, 4.1]","4.1 [3.5, 4.5]","2.8 [2.6, 3.5]","4.2 [4.0, 4.5]","3.7 [3.5, 3.8]","11.6 [11.2, 14.5]","77.1 [74.8, 93.1]","736.4 [725.9, 860.9]","9651.8 [9513.5, 9721.9]"
3,500,"5.1 [3.6, 6.3]","5.8 [5.2, 6.6]","7.8 [6.1, 8.5]","8.4 [7.5, 9.4]","5.1 [4.8, 6.7]","10.9 [9.4, 12.7]","7.4 [6.0, 9.6]","28.4 [27.3, 29.8]","228.1 [223.6, 238.4]","2893.3 [2856.0, 3234.1]","50731.2 [49428.7, 51878.9]"


Metodo,n,tiempo_Gamma0%,tiempo_Gamma10%,tiempo_Gamma100%,tiempo_Gamma25%,tiempo_Gamma5%,tiempo_Gamma50%,tiempo_Gamma75%,tiempo_SP1,tiempo_SP2,tiempo_SP3,tiempo_SP4
0,50,0.9 ± 0.5,0.9 ± 0.2,1.1 ± 0.3,1.1 ± 0.2,1.1 ± 0.4,1.3 ± 0.3,1.2 ± 0.3,8.5 ± 13.0,26.4 ± 7.9,198.4 ± 46.5,1480.6 ± 392.1
1,100,3.1 ± 1.5,2.6 ± 1.7,2.6 ± 0.9,3.4 ± 2.2,2.0 ± 0.6,2.5 ± 1.0,2.9 ± 1.3,7.6 ± 1.1,41.3 ± 5.7,358.0 ± 60.7,3350.7 ± 319.1
2,200,3.4 ± 1.6,3.0 ± 0.3,4.0 ± 1.0,4.4 ± 1.4,3.1 ± 0.8,4.4 ± 0.8,3.6 ± 0.3,12.9 ± 2.5,86.7 ± 17.3,812.9 ± 142.7,9601.3 ± 215.6
3,500,5.2 ± 1.7,6.3 ± 1.6,7.8 ± 2.1,9.3 ± 4.0,5.5 ± 1.1,11.2 ± 3.3,8.1 ± 2.5,29.0 ± 2.4,235.7 ± 20.5,3079.0 ± 323.9,50882.3 ± 1438.5


Metodo,n,tiempo_Gamma0%,tiempo_Gamma10%,tiempo_Gamma100%,tiempo_Gamma25%,tiempo_Gamma5%,tiempo_Gamma50%,tiempo_Gamma75%,tiempo_SP1,tiempo_SP2,tiempo_SP3,tiempo_SP4
0,50,0.0009 ± 0.0005,0.0009 ± 0.0002,0.0011 ± 0.0003,0.0011 ± 0.0002,0.0011 ± 0.0004,0.0013 ± 0.0003,0.0012 ± 0.0003,0.0085 ± 0.0130,0.0264 ± 0.0079,0.1984 ± 0.0465,1.4806 ± 0.3921
1,100,0.0031 ± 0.0015,0.0026 ± 0.0017,0.0026 ± 0.0009,0.0034 ± 0.0022,0.0020 ± 0.0006,0.0025 ± 0.0010,0.0029 ± 0.0013,0.0076 ± 0.0011,0.0413 ± 0.0057,0.3580 ± 0.0607,3.3507 ± 0.3191
2,200,0.0034 ± 0.0016,0.0030 ± 0.0003,0.0040 ± 0.0010,0.0044 ± 0.0014,0.0031 ± 0.0008,0.0044 ± 0.0008,0.0036 ± 0.0003,0.0129 ± 0.0025,0.0867 ± 0.0173,0.8129 ± 0.1427,9.6013 ± 0.2156
3,500,0.0052 ± 0.0017,0.0063 ± 0.0016,0.0078 ± 0.0021,0.0093 ± 0.0040,0.0055 ± 0.0011,0.0112 ± 0.0033,0.0081 ± 0.0025,0.0290 ± 0.0024,0.2357 ± 0.0205,3.0790 ± 0.3239,50.8823 ± 1.4385


In [11]:
import pandas as pd
from google.colab import files

# Diccionario con nombre de hoja y DataFrame
tablas = {
    "Mediana_ms": tabla_mediana_ms,
    "Media_s": tabla_media_s,
    "Mediana_IQR_ms": tabla_mediana_iqr_ms,
    "Media_Desv_ms": tabla_media_desv_ms,
    "Media_Desv_s": tabla_media_desv_s,
}

# Guardar todas las tablas en un Excel con una hoja cada una
with pd.ExcelWriter("anexo_tablas.xlsx") as writer:
    for nombre, df in tablas.items():
        df.to_excel(writer, sheet_name=nombre, index=False)

# Descargar el archivo
files.download("anexo_tablas.xlsx")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
df_stats_time_med.to_excel("df_stats_time_med.xlsx", index=False)
df_stats_time_mean.to_excel("df_stats_time_mean.xlsx", index=False)
files.download("df_stats_time_med.xlsx")
files.download("df_stats_time_mean.xlsx")

NameError: name 'df_stats_time_med' is not defined

In [ ]:
from google.colab import files

# df_stats_time.to_excel("df_stats_time.xlsx", index=False)
# files.download("df_stats_time.xlsx")
# wide_time.to_excel("wide_time.xlsx", index=False)
# files.download("wide_time.xlsx")

In [ ]:
from scipy.stats import shapiro
import pandas as pd

# Aplicamos test de Shapiro-Wilk por (n, Metodo)
def test_normalidad(grupo):
    stat, p = shapiro(grupo["Tiempo"])
    return pd.Series({
        "W-statistic": stat,
        "p-value": p,
        "Normal?": p > 0.05  # H0: los datos provienen de una dist. normal
    })

df_normalidad = long_time.groupby(["n", "Metodo"]).apply(test_normalidad).reset_index()

print(df_normalidad)


In [ ]:



for n, _ in wide_time.groupby("n")
cols_sp = ["tiempo_SP1", "tiempo_SP2", "tiempo_SP3", "tiempo_SP4"]
cols_bs = [c for c in wide_time.columns if c.startswith("tiempo_Gamma")]
cols = cols_sp + cols_bs

temp = wide_time[cols]
#display(temp)


long_time = temp.melt(var_name="Metodo", value_name="Tiempo")
#display(long_time)


# # --- Paso 4: Calcular estadísticas ---
def resumen(grupo):
    med = grupo.median()
    q1, q3 = grupo.quantile([0.25, 0.75])
    iqr = q3 - q1
    mean = grupo.mean()
    std = grupo.std()
    return pd.Series({
        "Median": med,
        "Median - IQR/2": med - iqr/2,
        "Median + IQR/2": med + iqr/2,
        "Mean": mean,
        "Mean - Std/2": mean - std/2,
        "Mean + Std/2": mean + std/2
    })

df_stats_time = long_time.groupby("Metodo")["Tiempo"].apply(resumen).reset_index()

print(df_stats_time)


In [ ]:
display(df_stats_time)

# ARCHIVED

In [ ]:
# --- Guardar a Excel (dos hojas) y descargar si estás en Colab ---
# out_name = "Resultados_SP1_SP4_n50_100_200.xlsx"
# with pd.ExcelWriter(out_name, engine="xlsxwriter") as writer:
#     df_res.to_excel(writer, sheet_name="long", index=False)
#     wide_sp.to_excel(writer, sheet_name="wide", index=False)

# if IN_COLAB:
#     files.download(out_name)
# else:
#     print(f"\n[Info] Archivo guardado: {out_name} (ejecuta en Colab para descargar automáticamente).")

# # --- Guardar a Excel (dos hojas) y descargar si estás en Colab ---
# out_name = "Resultados_SP1_SP4_n50_100_200.xlsx"
# with pd.ExcelWriter(out_name, engine="xlsxwriter") as writer:
#     df_res.to_excel(writer, sheet_name="long", index=False)
#     wide_sp.to_excel(writer, sheet_name="wide", index=False)

# if IN_COLAB:
#     files.download(out_name)
# else:
#     print(f"\n[Info] Archivo guardado: {out_name} (ejecuta en Colab para descargar automáticamente).")






# import numpy as np
# # PRUEBAS CON SOLO 1 INSTANCIA
# # --- Selecciona una instancia de df_kplib ---
# i = 6  # <-- cambia el indice si quieres otra fila
# u   = df_kplib.at[i, "Utilidades"]
# w   = df_kplib.at[i, "Pesos"]
# cap = float(df_kplib.at[i, "Capacidad"])
# arc = df_kplib.at[i, "Archivo"] if "Archivo" in df_kplib.columns else f"idx_{i}"

# bar_c, pesos = np.array(u, float), np.array(w, float)
# assert bar_c.size == pesos.size, "Utilidades y pesos deben tener la misma longitud"

# # --- Parametros de incertidumbre y muestreo ---
# pct_desviacion, seed, n_smpl = 0.50, 123, 8  # ajusta si quieres

# print(f"Instancia: {arc} | n={bar_c.size} | cap={cap} | pct={pct_desviacion} | n_smpl={n_smpl} | seed={seed}")

# for t in (1, 2, 3, 4):
#     print("\n" + "-"*60)
#     print(f"Resolviendo SP{t} ...")
#     sol = solve_sp(bar_c, pesos, cap, pct_desviacion, t=t, n_smpl=n_smpl, seed=seed, verbose=True)
#     print(f"Resumen SP{t}: status={sol['status']} | obj={sol['objective']:.4f} | vars={sol['n_vars']} | cons={sol['n_constrs']}")